In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
driver = webdriver.Chrome()
driver.get('http://www.portaltransparencia.gov.br/')

# Acessa pagina dos servidores
driver.find_element_by_id('servidores-card').click()
aTagsInLi = driver.find_elements_by_css_selector('li a')
for a in aTagsInLi:
    if a.get_attribute('href') == 'http://www.portaltransparencia.gov.br/servidores/lista-consultas':
        a.click()
        break

# Seleciona consulta por vinculo 
driver.find_element_by_xpath('/html/body/main/div[2]/div/div/ul/li[2]/a').click()

#Seleciona os servidores do tipo militar e civil
driver.find_element_by_xpath('//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/button').click()

try:
    militar = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/div/div/div[2]/div[2]/ul/li[3]/a/label/input'))
    )
    civil = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/div/div/div[2]/div[2]/ul/li[4]/a/label/input'))
    )
    
finally:
    driver.find_element_by_xpath('//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/div/div/div[2]/div[2]/ul/li[3]/a/label/input').click()
    driver.find_element_by_xpath('//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/div/div/div[2]/div[2]/ul/li[4]/a/label/input').click()
    driver.find_element_by_xpath('//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/div/div/div[2]/div[2]/ul/li[2]/input').click()

driver.find_element_by_class_name('botao__gera_paginacao_completa').click()


## Retirando dados de cada servidor
aTagsInTd = driver.find_elements_by_css_selector('td a')


pagina_servidores = []
for a in aTagsInTd:
    pagina_servidores.append(a.get_attribute('href'))
    
    
for pagina in pagina_servidores:
    print(pagina)
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(pagina)
    
    
    print("Current Page Title is : " + driver.title)
    driver.close()

    driver.switch_to.window(driver.window_handles[0])
    print("Current Page Title is : " + driver.title)
       


In [ ]:
print